# Modelo de Chandrasekhar de una Enana Blanca

In [ ]:
# Importamos las librerías necesarias
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
plt.rcParams['figure.figsize'] = (8,6) # Tamaño de las figuras
plt.rcParams['axes.labelsize'] = 14 # Tamaño de las etiquetas de los ejes
plt.rcParams['xtick.labelsize'] = 10 # Tamaño de las marcas de los ejes
plt.rcParams['ytick.labelsize'] = 10 # Tamaño de las marcas de los ejes
plt.rcParams['font.size'] = 15 # Tamaño de la fuente del titulo
from scipy.integrate import odeint

Para un gas fermiónico completamente degenerado con función de ocupación:

\begin{equation*}
f(\epsilon) = \lim_{T \to 0} \frac{1}{e^{\frac{\epsilon - \mu}{kT}} + 1} =
 \left\{
\begin{array}{ll}
1 & \text{si } \epsilon < \epsilon_F \\
0 & \text{si } \epsilon > \epsilon_F
\end{array}
\right.,
\end{equation*}

las integrales de densidad de electrones, presión y densidad de energía de una enana blanca son:

\begin{align*}
n_e &= \frac{8 \pi}{h^3} \int_0^{p_F} p^2 dp = \frac{8 \pi}{3 h^3} p_F^3, \\
P &= \frac{8 \pi}{3h^3} \int_0^{p_F} \frac{p^4 c^2 dp}{\sqrt{p^2 c^2 + m^2 c^4}}, \\ 
\varepsilon &= \frac{8 \pi}{h^3} \int_0^{p_F} \sqrt{p^2 c^2 + m^2 c^4} p^2 dp.
\end{align*}

En el caso no relativista $\frac{p_F}{m c} \ll 1$, la presión se puede aproximar como:

\begin{equation*}
P = \frac{8 \pi}{3h^3} \int_0^{p_F} \frac{p^4dp}{m} = \frac{8 \pi}{15 h^3} \frac{p_F^5}{m},
\end{equation*}

la cual se puede relacionar con la densidad de masa de la estrella considerando $\rho = m_n \mathcal{k} n_e$:

\begin{equation*}
P = \left( \frac{3}{8\pi} \right)^{2/3} \frac{h^2}{5m_e(m_n \mathcal{k})^{5/3}} \rho^{5/3}.
\end{equation*}

Para el caso altamente relativista $\frac{p_F}{m c} \gg 1$, la presión se puede aproximar como:

\begin{equation*}
P = \frac{8 \pi}{3h^3} \int_0^{p_F} c p^3 dp = \frac{8 \pi}{12 h^3} p_F^4 c,
\end{equation*}

y, en términos de la densidad de masa:

\begin{equation*}
P = \left( \frac{3}{8\pi} \right)^{1/3} \frac{h c}{4(m_n \mathcal{k})^{4/3}} \rho^{4/3}.
\end{equation*}

Estas expresiones son ecuaciones de estado politrópicas, las cuales se pueden utilizar para modelar la estructura de una enana blanca.

In [ ]:
# Definimos las constantes
k = 2 # Constante de proporcionalidad nucleones por electrón
m_n = 1.67e-27 # Masa de los nucleones en kg
m_e = 9.11e-31 # Masa de los electrones en kg
h = 6.63e-34 # Constante de Planck en J s
c = 3e8 # Velocidad de la luz en m/s
pi = np.pi # Valor de pi

Para poder resolver el sistema, realizamos las siguientes sustituciones de variables:

\begin{align*}
    \rho(r) &= \rho_0 \theta^n, \\
    P(r) &= K \rho(r)^{\gamma}, \\
    \alpha^2 &= \frac{(n+1)K}{4\pi G} \rho_0^{n^{-1}-1}, \\
    \xi &= \frac{r}{\alpha},
\end{align*}

de forma que el sistema se reduce a la ecuación de Lane-Emden:

\begin{equation*}
    \frac{1}{\xi^2} \frac{d}{d\xi} \left( \xi^2 \frac{d\theta}{d\xi} \right) + \theta^n = 0.
\end{equation*}

En este caso se tienen las condiciones iniciales:

\begin{align*}
    \theta(0) &= 1, \\
    \theta'(0) &= 0.
\end{align*}

Sin embargo, por la forma de la ecuación, no se puede iniciar en $\xi = 0$, por lo que se inicia en $\xi = \epsilon$, para el cual las condiciones iniciales, al expandir las funciones y emplear las ecuaciones, son:

\begin{align*}
    \theta(\epsilon) &= 1 - \frac{1}{6} \theta_2 \epsilon^2 + \mathcal{O}(\epsilon^4), \\
    \theta'(\epsilon) &= -\frac{1}{3} \theta_2 \epsilon + \mathcal{O}(\epsilon^3),
\end{align*}

donde $\theta_2$ es una constante que se puede determinar a partir de la ecuación de Lane-Emden.